#**Recommendation Systems**

> In this notebook, the different recommendation systems available in the Cornac library will be executed.



#**1.- DATASET LOADING**

In [ ]:
# Download the  dataset in compressed JSON Lines format from a web archive URL
!wget -O Gift_Cards.jsonl.gz https://web.archive.org/web/20240314164222/https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/Gift_Cards.jsonl.gz --no-check-certificate

# Remove any existing uncompressed file named 'xxxxx.jsonl' to avoid conflicts
!rm -f Gift_Cards.jsonl

# Decompress the downloaded gzip file to obtain 'xxxxx.jsonl'
!gzip -d Gift_Cards.jsonl.gz


--2025-07-03 09:33:52--  https://web.archive.org/web/20240314164222/https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/Gift_Cards.jsonl.gz
Resolving web.archive.org (web.archive.org)... 207.241.237.3
Connecting to web.archive.org (web.archive.org)|207.241.237.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12556849 (12M) [application/x-gzip]
Saving to: ‘Gift_Cards.jsonl.gz’

Gift_Cards.jsonl.gz 100%[===================>]  11.97M  20.8MB/s    in 0.6s    

2025-07-03 09:33:54 (20.8 MB/s) - ‘Gift_Cards.jsonl.gz’ saved [12556849/12556849]



#**1.2.- PACKAGE LOADING**

In [ ]:
# Install the Cornac library
!pip install cornac

# Import the Cornac library and relevant modules for evaluation, models, and metrics
import cornac
from cornac.eval_methods import RatioSplit  # For splitting data into train/test sets
from cornac.models import MF, PMF, BPR, VAECF, COE, HPF, IBPR, OnlineIBPR  # Different recommendation algorithms
from cornac.metrics import FMeasure, Precision, Recall, NDCG, AUC, MAP  # Evaluation metrics for recommendation quality

# Import random module for any randomization needs in experiments or data processing
import random



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 15.3 MB/s eta 0:00:00


#**2.- CORNAC LIBRARY DATA LOADING**

In [ ]:
# Import the json module for reading JSON formatted data
import json
# Import pandas for data manipulation and analysis
import pandas as pd

# Define the filename containing the dataset
file = "Gift_Cards.jsonl"

# Initialize an empty list to store parsed data
all_data = []

# Open the JSON lines file and read it line by line
with open(file, 'r') as fp:
    for line in fp:
        # Parse each line as JSON and strip any extra whitespace
        line_data = json.loads(line.strip())
        # Append a tuple of (user_id, item id (asin), rating) to the list
        all_data.append((line_data["user_id"], line_data["asin"], line_data["rating"]))

# Convert the list of tuples into a pandas DataFrame with specified column names
all_data = pd.DataFrame(all_data, columns=["user_id", "item", "rating"])

# Display the first few rows of the DataFrame to verify data loading
all_data.head()

,user_id,item,rating
0,AHZ6XMOLEWA67S3TX7IWEXXGWSOA,B00IX1I3G6,5.0
1,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B005ESMMWW,5.0
2,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B01K8RIM5Y,5.0
3,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B0091JKVU0,5.0
4,AH5L7ILVA6HYLZOUZIQAWNHVVK3A,B00FTGTM5E,1.0


##  **2.1- DATASET DIVISION FOR MODELING**

In [ ]:
# Import RatioSplit evaluation method from Cornac library
from cornac.eval_methods import RatioSplit

# Set a fixed random seed to ensure reproducibility of the data split
random_seed = 2533

# Initialize RatioSplit to split the dataset into training, validation, and test sets
# test_size=0.2 means 20% of data will be used for testing
# val_size=0.2 means 20% of data will be used for validation
# rating_threshold=3.0 means ratings >= 3 are considered positive feedback
rs = RatioSplit(data=all_data.values, test_size=0.2, val_size=0.2, rating_threshold=3.0, seed=random_seed)

# Output the number of training samples after splitting
print(f"There are {rs.train_size} training samples")
# Output the number of validation samples after splitting
print(f"There are {rs.val_size} validation samples")
# Output the number of test samples after splitting
print(f"There are {rs.test_size} test samples")

/usr/local/lib/python3.11/dist-packages/cornac/data/dataset.py:335: UserWarning: 519 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.11/dist-packages/cornac/data/dataset.py:335: UserWarning: 26 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.11/dist-packages/cornac/data/dataset.py:335: UserWarning: 24 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


There are 91446 training samples
There are 30482 validation samples
There are 30482 test samples


In [ ]:
# Initialize evaluation metrics for the recommendation models

# FMeasure (F1 score) at cutoff 10: balances precision and recall for top 10 recommendations
F1_at_10 = cornac.metrics.FMeasure(k=10)

# FMeasure at cutoff 100: evaluates F1 score for top 100 recommendations
F1_at_100 = cornac.metrics.FMeasure(k=100)

# Precision at cutoff 50: measures proportion of relevant items in the top 50 recommendations
Precision = cornac.metrics.Precision(k=50)

# Recall at cutoff 50: measures how many relevant items are retrieved within top 50 recommendations
Recall = cornac.metrics.Recall(k=50)

# Normalized Discounted Cumulative Gain: accounts for the position of relevant items in the ranking
NDCG = cornac.metrics.NDCG()

# Area Under the ROC Curve: evaluates the model’s ability to distinguish between positive and negative interactions
AUC = cornac.metrics.AUC()

# Mean Average Precision: measures average precision across all users
MAP = cornac.metrics.MAP()

# Mean Absolute Error: average absolute difference between predicted and true ratings (for rating prediction)
MAE = cornac.metrics.MAE()

# Root Mean Squared Error: square root of average squared difference between predicted and true ratings
RMSE = cornac.metrics.RMSE()

# Mean Squared Error: average squared difference between predicted and true ratings
MSE = cornac.metrics.MSE()

#**3.- POP**

In [ ]:
# Define the recommendation model to use: MostPop recommends the most popular items to all users
most_pop = cornac.models.MostPop()

# Set up and run the experiment
# - eval_method: data splitting method defined earlier (rs)
# - models: list of models to evaluate (here only MostPop)
# - metrics: evaluation metrics to measure performance (Recall, NDCG, AUC)
# - verbose=True: print progress and results during execution
# - user_based=True: evaluation is performed from a user perspective
cornac.Experiment(
    eval_method=rs,
    models=[most_pop],
    metrics=[Recall, NDCG, AUC],
    verbose=True,
    user_based=True
).run()


[MostPop] Training started!

[MostPop] Evaluation started!


Ranking:   0%|          | 0/4120 [00:00<?, ?it/s]

Ranking:   0%|          | 0/4030 [00:00<?, ?it/s]


VALIDATION:
...
        |    AUC | NDCG@-1 | Recall@50 | Time (s)
------- + ------ + ------- + --------- + --------
MostPop | 0.9113 |  0.2881 |    0.4903 |   2.9032

TEST:
...
        |    AUC | NDCG@-1 | Recall@50 | Train (s) | Test (s)
------- + ------ + ------- + --------- + --------- + --------
MostPop | 0.9136 |  0.2843 |    0.5030 |    0.0230 |   3.4817



#**4.- MF**

In [ ]:
import cornac
from cornac.hyperopt import Discrete, Continuous, GridSearch

# Define the matrix factorization (MF) model with some default hyperparameters
MF = cornac.models.MF(
    k=10,               # Number of latent factors
    max_iter=100,       # Maximum iterations for training
    learning_rate=0.01, # Learning rate for optimization
    lambda_reg=0.02,    # Regularization term to prevent overfitting
    use_bias=True,      # Include user/item biases
    seed=random_seed    # Seed for reproducibility
)

# Define the hyperparameter search space for grid search
space = [
    Discrete(name="k", values=[5, 15, 45, 100]),             # Possible values for latent factors
    Discrete(name="learning_rate", values=[0.001, 0.01, 0.1, 0.5])  # Possible learning rates
]

# Set up grid search to tune MF hyperparameters by minimizing MAE metric
gs_MF = GridSearch(
    model=MF,
    space=space,
    metric=MAE,
    eval_method=rs  # Use the same data split method defined earlier
)

# Run the experiment with the grid search model
cornac.Experiment(
    eval_method=rs,
    models=[gs_MF],              # Grid search will handle multiple model trainings internally
    metrics=[MAE, RMSE, MSE],   # Metrics to evaluate performance during tuning
    verbose=True,               # Show training progress and results
    user_based=True             # Evaluate from user perspective
).run()

# Print the best hyperparameters found by the grid search
print("best hyperparameters:", gs_MF.best_params)


[GridSearch_MF] Training started!
Evaluating: {'k': 5, 'learning_rate': 0.001}
Evaluating: {'k': 5, 'learning_rate': 0.01}
Evaluating: {'k': 5, 'learning_rate': 0.1}
Evaluating: {'k': 5, 'learning_rate': 0.5}
Evaluating: {'k': 15, 'learning_rate': 0.001}
Evaluating: {'k': 15, 'learning_rate': 0.01}
Evaluating: {'k': 15, 'learning_rate': 0.1}
Evaluating: {'k': 15, 'learning_rate': 0.5}
Evaluating: {'k': 45, 'learning_rate': 0.001}
Evaluating: {'k': 45, 'learning_rate': 0.01}
Evaluating: {'k': 45, 'learning_rate': 0.1}
Evaluating: {'k': 45, 'learning_rate': 0.5}
Evaluating: {'k': 100, 'learning_rate': 0.001}
Evaluating: {'k': 100, 'learning_rate': 0.01}
Evaluating: {'k': 100, 'learning_rate': 0.1}
Evaluating: {'k': 100, 'learning_rate': 0.5}
Best parameter settings: {'k': 5, 'learning_rate': 0.1}
MAE = 0.2468

[GridSearch_MF] Evaluation started!


Rating:   0%|          | 0/4753 [00:00<?, ?it/s]

Rating:   0%|          | 0/4673 [00:00<?, ?it/s]


VALIDATION:
...
              |    MAE |    MSE |   RMSE | Time (s)
------------- + ------ + ------ + ------ + --------
GridSearch_MF | 0.2515 | 0.3301 | 0.2565 |   1.6679

TEST:
...
              |    MAE |    MSE |   RMSE | Train (s) | Test (s)
------------- + ------ + ------ + ------ + --------- + --------
GridSearch_MF | 0.2636 | 0.3762 | 0.2685 |   16.6169 |   1.8991

best hyperparameters: {'k': 5, 'learning_rate': 0.1}


# **5.- BPR**

In [ ]:
# Define Bayesian Personalized Ranking (BPR) model with initial hyperparameters
BPR = cornac.models.BPR(
    k=100,               # Number of latent factors
    max_iter=100,        # Maximum iterations for training
    learning_rate=0.5,   # Learning rate for optimization
    lambda_reg=0.02,     # Regularization term to avoid overfitting
    seed=random_seed     # Seed for reproducibility
)

# Define hyperparameter search space for lambda_reg (regularization parameter)
space = [
    Discrete(name="lambda_reg", values=[0.0001, 0.001, 0.01, 0.1, 0.5])
]

# Set up grid search to tune lambda_reg hyperparameter optimizing for NDCG metric
gs_BPR = GridSearch(
    model=BPR,
    space=space,
    metric=NDCG,
    eval_method=rs  # Use the same data split defined previously
)

# Run the experiment with grid search applied to BPR model
cornac.Experiment(
    eval_method=rs,
    models=[gs_BPR],           # Grid search manages multiple trainings internally
    metrics=[Recall, NDCG, AUC],  # Metrics to evaluate model performance
    verbose=True,              # Show detailed output during training
    user_based=True            # Evaluate from user perspective
).run()

# Output the best regularization hyperparameter found by grid search
print("best hyperparameters:", gs_BPR.best_params)


[GridSearch_BPR] Training started!
Evaluating: {'lambda_reg': 0.0001}
Evaluating: {'lambda_reg': 0.001}
Evaluating: {'lambda_reg': 0.01}
Evaluating: {'lambda_reg': 0.1}
Evaluating: {'lambda_reg': 0.5}
Best parameter settings: {'lambda_reg': 0.01}
NDCG@-1 = 0.3399

[GridSearch_BPR] Evaluation started!


Ranking:   0%|          | 0/4120 [00:00<?, ?it/s]

Ranking:   0%|          | 0/4030 [00:00<?, ?it/s]


VALIDATION:
...
               |    AUC | NDCG@-1 | Recall@50 | Time (s)
-------------- + ------ + ------- + --------- + --------
GridSearch_BPR | 0.8986 |  0.3389 |    0.5577 |   7.1321

TEST:
...
               |    AUC | NDCG@-1 | Recall@50 | Train (s) | Test (s)
-------------- + ------ + ------- + --------- + --------- + --------
GridSearch_BPR | 0.9004 |  0.3387 |    0.5688 |   42.8734 |   4.3865

best hyperparameters: {'lambda_reg': 0.01}


# **6.-GA**

In [ ]:
# Define the Global Average model, which predicts ratings using the overall average rating in the dataset
global_avg = cornac.models.GlobalAvg()

# Set up and run the experiment using the RatioSplit evaluation method
cornac.Experiment(
    eval_method=rs,             # Use previously defined train/val/test split
    models=[global_avg],        # Evaluate the Global Average model
    metrics=[MAE, RMSE, MSE],   # Evaluate with error metrics: Mean Absolute Error, Root Mean Squared Error, Mean Squared Error
    verbose=True,               # Print detailed progress and results
    user_based=True             # Evaluate recommendations from the user perspective
).run()


[GlobalAvg] Training started!

[GlobalAvg] Evaluation started!


Rating:   0%|          | 0/4753 [00:00<?, ?it/s]

Rating:   0%|          | 0/4673 [00:00<?, ?it/s]


VALIDATION:
...
          |    MAE |    MSE |   RMSE | Time (s)
--------- + ------ + ------ + ------ + --------
GlobalAvg | 0.5684 | 0.6223 | 0.5699 |   1.6946

TEST:
...
          |    MAE |    MSE |   RMSE | Train (s) | Test (s)
--------- + ------ + ------ + ------ + --------- + --------
GlobalAvg | 0.5725 | 0.6493 | 0.5735 |    0.0005 |   1.4026



# **7.- NeuMF**

In [ ]:
# Set parameters for the NeuMF (Neural Matrix Factorization) model
num_factors = 16             # Number of latent factors for embeddings
layers = [64, 32, 16, 8]     # Sizes of the hidden layers in the neural network
act_fn = "relu"              # Activation function used in hidden layers
learner = "adam"             # Optimization algorithm
backend = "pytorch"          # Deep learning backend: PyTorch (alternative is TensorFlow)
num_epochs = 20              # Number of training epochs
batch_size = 256             # Number of samples per training batch
lr = 0.01                    # Learning rate for optimization
num_neg = 10                 # Number of negative samples per positive instance for training
seed = random_seed           # Seed for reproducibility

# Initialize the NeuMF model with the specified parameters
neumf = cornac.models.NeuMF(
    num_factors=num_factors,
    layers=layers,
    act_fn=act_fn,
    learner=learner,
    backend=backend,
    num_epochs=num_epochs,
    batch_size=batch_size,
    lr=lr,
    num_neg=num_neg,
    seed=seed
)

# Run the experiment using the defined evaluation method and metrics
cornac.Experiment(
    eval_method=rs,                # Use the predefined train/val/test splits
    models=[neumf],               # Evaluate the NeuMF model
    metrics=[Recall, NDCG, AUC],  # Metrics to assess recommendation quality
    verbose=True                  # Enable detailed output during training
).run()


[NeuMF] Training started!


  0%|          | 0/20 [00:00<?, ?it/s]


[NeuMF] Evaluation started!


Ranking:   0%|          | 0/4120 [00:00<?, ?it/s]

Ranking:   0%|          | 0/4030 [00:00<?, ?it/s]


VALIDATION:
...
      |    AUC | NDCG@-1 | Recall@50 | Time (s)
----- + ------ + ------- + --------- + --------
NeuMF | 0.7874 |  0.2457 |    0.3153 |   7.2499

TEST:
...
      |    AUC | NDCG@-1 | Recall@50 | Train (s) | Test (s)
----- + ------ + ------- + --------- + --------- + --------
NeuMF | 0.7928 |  0.2462 |    0.3193 |  517.1325 |   8.4291



#**8.- NEURAL RATING**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from cornac.models import Recommender

# Define a custom neural network-based recommender model inheriting from Cornac's Recommender class
class Neural_Rating(Recommender):
    def __init__(self, embedding_dim=10, dropout=0.2, dense_units1=10,
                 dense_units2=8, l2_reg=0.001,
                 epochs=50, batch_size=64,
                 learning_rate=0.0001, patience=5,
                 verbose=1, **kwargs):
        super().__init__(name="NR", **kwargs)  # Initialize parent class with model name "NR"

        # Set hyperparameters for model architecture and training
        self.embedding_dim = embedding_dim        # Dimensionality of user/item embeddings
        self.dropout = dropout                    # Dropout rate for regularization
        self.dense_units1 = dense_units1          # Units in first dense layer
        self.dense_units2 = dense_units2          # Units in second dense layer
        self.l2_reg = l2_reg                      # L2 regularization factor for weights
        self.epochs = epochs                      # Number of training epochs
        self.batch_size = batch_size              # Batch size for training
        self.learning_rate = learning_rate        # Learning rate for optimizer
        self.patience = patience                  # Early stopping patience
        self.verbose = verbose                    # Verbosity level during training
        self.model = None                         # Placeholder for Keras model instance

    def fit(self, train_set, val_set=None):
        # Call parent class fit to initialize training and validation sets
        Recommender.fit(self, train_set, val_set)

        # Retrieve total number of users and items from training data
        num_users, num_items = train_set.num_users, train_set.num_items

        # Input layers for user IDs and item IDs
        user_input = Input(shape=(1,), name='user_input')
        item_input = Input(shape=(1,), name='item_input')

        # User and item embedding layers with L2 regularization
        user_embedding = Embedding(input_dim=num_users, output_dim=self.embedding_dim,
                                   input_length=1, name='user_embedding',
                                   embeddings_regularizer=l2(self.l2_reg))(user_input)
        item_embedding = Embedding(input_dim=num_items, output_dim=self.embedding_dim,
                                   input_length=1, name='item_embedding',
                                   embeddings_regularizer=l2(self.l2_reg))(item_input)

        # Flatten embedding outputs to feed into dense layers
        user_flatten = Flatten()(user_embedding)
        item_flatten = Flatten()(item_embedding)

        # Concatenate user and item embeddings into one feature vector
        concat = Concatenate()([user_flatten, item_flatten])

        # First dense layer with ReLU activation, batch normalization, and dropout
        dense1 = Dense(self.dense_units1, activation='relu',
                       kernel_regularizer=l2(self.l2_reg))(concat)
        batch_norm1 = BatchNormalization()(dense1)
        dropout1 = Dropout(self.dropout)(batch_norm1)

        # Second dense layer with ReLU activation, batch normalization, and dropout
        dense2 = Dense(self.dense_units2, activation='relu',
                       kernel_regularizer=l2(self.l2_reg))(dropout1)
        batch_norm2 = BatchNormalization()(dense2)
        dropout2 = Dropout(self.dropout)(batch_norm2)

        # Output layer with linear activation predicting the rating
        output = Dense(1, activation='linear')(dropout2)

        # Define the full Keras model mapping user and item inputs to predicted rating
        self.model = Model(inputs=[user_input, item_input],
                           outputs=output)

        # Compile the model with Adam optimizer and mean squared error loss function
        self.model.compile(optimizer=Adam(learning_rate=self.learning_rate),
                           loss='mean_squared_error')

        # Extract user IDs, item IDs, and ratings from the training set
        user_ids, item_ids, ratings = train_set.uir_tuple
        user_ids = np.array(user_ids, dtype=np.int64)
        item_ids = np.array(item_ids, dtype=np.int64)
        ratings = np.array(ratings, dtype=np.float32)

        # Setup early stopping callback to prevent overfitting
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=self.patience,
                                                      restore_best_weights=True)]

        # If validation set is provided, prepare validation data and fit the model with validation
        if val_set is not None:
            val_user_ids, val_item_ids, val_ratings = val_set.uir_tuple
            val_user_ids = np.array(val_user_ids, dtype=np.int64)
            val_item_ids = np.array(val_item_ids, dtype=np.int64)
            val_ratings = np.array(val_ratings, dtype=np.float32)

            self.model.fit([user_ids, item_ids],
                           ratings,
                           validation_data=([val_user_ids, val_item_ids], val_ratings),
                           epochs=self.epochs,
                           batch_size=self.batch_size,
                           verbose=self.verbose, callbacks=callbacks)
        else:
            # Fit model without validation data if no validation set is given
            self.model.fit([user_ids, item_ids], ratings, epochs=self.epochs,
                           batch_size=self.batch_size,
                           verbose=self.verbose,
                           callbacks=callbacks)

        return self

    def score(self, user_idx, item_idx=None):
        # If predictions are not precomputed, generate predictions for all user-item pairs
        if not hasattr(self, 'predictions'):
            user_indices = np.repeat(np.arange(self.train_set.num_users),
                                     self.train_set.num_items)
            item_indices = np.tile(np.arange(self.train_set.num_items),
                                   self.train_set.num_users)

            # Predict ratings in batch and reshape to user x item matrix
            self.predictions = self.model.predict([user_indices, item_indices],
                                                  batch_size=2048, verbose=0).reshape(self.train_set.num_users,
                                                                                      self.train_set.num_items)

        # Return predictions for a single user (all items) or for specific user-item pair
        if item_idx is None:
            return self.predictions[user_idx, :]
        else:
            return self.predictions[user_idx, item_idx]

In [ ]:
# Initiate the custom Neural_Rating recommender model with default parameters
NR = Neural_Rating()

# Set up a Cornac experiment using the previously defined RatioSplit evaluation method (rs)
# Include the custom model NR in the list of models to evaluate
# Use MAE, RMSE, and MSE as evaluation metrics to assess prediction accuracy
# Enable verbose mode for detailed output during training and evaluation
# Specify user_based=True to indicate evaluation per user
experiment = cornac.Experiment(
    eval_method=rs,
    models=[NR],
    metrics=[MAE, RMSE, MSE],
    verbose=True,
    user_based=True
)

# Run the experiment: this will train NR on the training data,
# validate on the validation set, and evaluate performance on the test set
experiment.run()


[NR] Training started!


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 27s 17ms/step - loss: 22.5568 - val_loss: 17.8412
Epoch 2/50
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 22s 16ms/step - loss: 15.1723 - val_loss: 10.5601
Epoch 3/50
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - loss: 8.5393 - val_loss: 4.5799
Epoch 4/50
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 42s 16ms/step - loss: 4.2665 - val_loss: 1.6693
Epoch 5/50
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - loss: 2.6759 - val_loss: 0.9737
Epoch 6/50
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 24s 17ms/step - loss: 2.2978 - val_loss: 0.8843
Epoch 7/50
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 39s 16ms/step - loss: 2.0496 - val_loss: 0.8377
Epoch 8/50
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 22s 16ms/step - loss: 1.8719 - val_loss: 0.8199
Epoch 9/50
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 43s 17ms/step - loss: 1.7240 - val_loss: 0.7706
Epoch 10/50
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 23s 16ms/step - loss: 1.5804 - val_loss: 0.7214
Epoch 11/50
1421/1421 ━━━━━━━━━━━━━━━━━━━━ 42s 17ms/step - loss: 1.4905 - val_loss: 0.7067
Epoc

Rating:   0%|          | 0/4753 [00:00<?, ?it/s]

Rating:   0%|          | 0/4673 [00:00<?, ?it/s]


VALIDATION:
...
   |    MAE |    MSE |   RMSE | Time (s)
-- + ------ + ------ + ------ + --------
NR | 0.2980 | 0.3393 | 0.3022 |   1.3883

TEST:
...
   |    MAE |    MSE |   RMSE | Train (s) | Test (s)
-- + ------ + ------ + ------ + --------- + --------
NR | 0.3216 | 0.4139 | 0.3261 |  857.0685 | 279.9042

